In [1]:
# import packages
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import StaleElementReferenceException
import time 
import os
import json
import urllib


In [2]:
# Some urls are blocked from untrusted sites, the below code make python environment to trust every site
import os, ssl
if (not os.environ.get('PYTHONHTTPSVERIFY', '') and
getattr(ssl, '_create_unverified_context', None)):
    ssl._create_default_https_context = ssl._create_unverified_context

In [7]:
def download_images():
    
    f = open('site_urls.txt','r')
    search_url = f.readlines()[25]
    print(search_url)
   
    # path of chorme drive to run selenium
    path = r'chromedriver'
    
    driver = webdriver.Chrome(path)
    
    k = open('wildlife keywords.txt','r')
    kw = k.readlines()
    
    for i in range(0,len(kw)):
        print(i)
        keyW = kw[i]
        
        driver.get(search_url)

        time.sleep(2)
        
        # search text box 
        box = driver.find_element_by_xpath('//*[@id="inline-search-box"]')  
        box.clear()
        #sending the keyword
        box.send_keys(keyW)
        time.sleep(2)
        
         # try block if there are no ad results for the keyword it will continue with next keyword
        try:
            # collecting links of ad results
            links = driver.find_element_by_xpath('//*[@id="products_search_results"]')
            sub_lnks = links.find_elements_by_tag_name('a')

            lnks_lst = []

            for lnk in sub_lnks:
                lnks_lst.append(lnk.get_attribute('href'))
            print(lnks_lst[0:3])

            # making a directory like URL+Keyword_number , example 25_0 , 25_1,...
            if lnks_lst:
                if not os.path.exists(str(25)+'_'+str(i)):
                    os.mkdir(str(25)+'_'+str(i))
                    
            #  iterating through ads results
            for j in range(0,2):
                driver.get(lnks_lst[j])
                print(lnks_lst[j])
                time.sleep(2)
                # Title capturing 
                sub_txt = driver.find_element_by_xpath('//*[@id="product-description"]/h1').text
                
                #making directory inside the above created directory like 0 means first ad, 1 for second ad....
                if not os.path.exists(str(25)+'_'+str(i) +'/'+ str(j)):
                    os.mkdir(str(25)+'_'+str(i) +'/'+ str(j))
                    
                # making dictionary
                details = {
                    'name' : sub_txt,
                    'price' : driver.find_element_by_xpath('//*[@id="product-price"]/span[1]').text,
                    'location': None,
                    'details': None,
                    'description' : driver.find_element_by_xpath('//*[@id="product-description-short"]').get_attribute("innerText"),
                    'url' : search_url
                }
                
                # writing the dictionary to text file like 0.txt for first ad ....
                with open(str(25)+'_'+str(i) +'/'+ str(j) + '/' + str(j) + '.txt', 'w') as tfile:
                     tfile.write(json.dumps(details))
                
                # collecting image urls
                image_lst = []
                elem1 = driver.find_element_by_xpath('//*[@id="productCarousel"]/ul')
                sub = elem1.find_elements_by_tag_name('img')
                for k,l in enumerate(sub):
                    src = l.get_attribute('data-src')
                    image_lst.append(src)
                    print(src)
                    
                    # collecting images and saving it inside the above created directory like 0_0.jpg for first ad first image
                    # 0_1.jpg for first ad second image ...
                    if src != None:
                        urllib.request.urlretrieve(str(src), os.path.join(str(25)+'_'+str(i), str(j), str(j)+'_'+str(k)+'.jpg'))
                # writing image urls to text file like 0_imgs.txt for first ad ....
                with open(str(25)+'_'+str(i) +'/'+ str(j) + '/' + str(j) + '_imgs.txt', 'w') as tfile:
                     tfile.write(json.dumps(image_lst)) 
        except:
            continue
            
    driver.close()
    

In [8]:
def main():
    download_images()
    
if __name__ == "__main__":
    main()

http://www.ioffer.com/search/items/[KW]?order=price_desc

0
['https://www.ioffer.com/vp/sylvie-mini-alligator-top-handle-bag-white-147032', 'https://www.ioffer.com/vp/alligator-sunglasses-4208-sunglasses-original-box-2751016', 'https://www.ioffer.com/vp/alligator-sunglasses-2501-sunglasses-original-box-2751222']
https://www.ioffer.com/vp/sylvie-mini-alligator-top-handle-bag-white-147032
https://ioffer-assets.oss-eu-west-1.aliyuncs.com/as/variants/emqy30oz677/i77fbw1jgz/tvyiumb/12/e469c2f40f/c4881820e0/ef2799e5f3/48067a6be9/5208753628/72d1b3484d/e6
https://ioffer-assets.oss-eu-west-1.aliyuncs.com/as/variants/mdzj931n0ie/d9kwpzecra/ffgrogv/12/e469c2f40f/c4881820e0/ef2799e5f3/48067a6be9/5208753628/72d1b3484d/e6
https://ioffer-assets.oss-eu-west-1.aliyuncs.com/as/variants/sqf5iwzrpsc/p64rlcvwad/nlsgau4/12/e469c2f40f/c4881820e0/ef2799e5f3/48067a6be9/5208753628/72d1b3484d/e6
https://www.ioffer.com/vp/alligator-sunglasses-4208-sunglasses-original-box-2751016
https://ioffer-assets.oss-eu-west-